# Creating a Simple Agent with Tracing

In [1]:
import dotenv
import os

from openai import OpenAI

dotenv.load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    print(
        """Error: OPENAI_API_KEY environment variable not set. Please copy the .env.template file as .env and fill it in.
    
    You can execute these commands in the terminal to get started:
    cp .env.template .env
    code .env
    """
    )

# Test OpenAI Access
print(
    OpenAI()
    .responses.create(
        model=os.environ["OPENAI_DEFAULT_MODEL"], input="Say: We are up and running!"
    )
    .output_text
)

We are up and running!


In [3]:
from agents import Agent, Runner, trace
from openai.types.responses import ResponseTextDeltaEvent

Create a simple Nutrition Assistant Agent

In [4]:
nutrition_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful assisant giving out nutrition advice.
    You give concise answers.
    """
)

Let's execute the Agent:

In [5]:
with trace("Simple Nutrition Agent"):
    result = await Runner.run(nutrition_agent, "How healthy are bananas?")

print(result)

RunResult:
- Last agent: Agent(name="Nutrition Assistant", ...)
- Final output (str):
    Bananas are healthy as part of a balanced diet. They’re rich in potassium, vitamin B6, vitamin C, and fiber, and are naturally low in fat. A medium banana (~105 kcal) fits well into most diets. They have moderate sugars and GI. Tips: pair with protein or healthy fats to slow sugar absorption; choose ripeness based on preference or goals (unripe for slower sugars, ripe for sweeter taste). Avoid if you have specific kidney or diabetes concerns without medical guidance.
- 2 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


Streaming the answer to the screen, token by token

In [6]:
response_stream = Runner.run_streamed(nutrition_agent, "How healthy are bananas?")

async for event in response_stream.stream_events():
    if event.type == "raw_response_event" and isinstance(
        event.data, ResponseTextDeltaEvent
    ):
        print(event.data.delta, end="", flush=True)

Overall, yes—bananas are healthy when eaten as part of a balanced diet.

Key points:
- Nutrients: good source of potassium, vitamin C, vitamin B6, and dietary fiber.
- Calories/macros: about 105 kcal per medium banana; ~27 g carbs; ~3 g fiber; low fat.
- Benefits: supports heart health, digestion, and convenient energy.
- Considerations: natural sugars; can impact blood sugar if eaten in large amounts—okay for most people, but diabetics should fit them into total carb intake. People with kidney disease may need to limit potassium.

How many? 1–2 per day is reasonable for most people.

_Good Job!_